In [ ]:
import pandas as pd
from ipywidgets import FileUpload, Button, VBox, Label
from IPython.display import display
from io import StringIO
from difflib import SequenceMatcher
import unicodedata

In [ ]:
# -----------------------
# Funções auxiliares
# -----------------------
def normalize_colname(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = s.strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = s.replace(" ", "_")
    return s

def find_column(cols, target_norm):
    for c in cols:
        if normalize_colname(c) == target_norm:
            return c
    return None

def safe_get_str(row, colname):
    """Retorna o valor da célula como string limpa, ou '' se NaN/None."""
    val = row.get(colname, "")
    if pd.isna(val):
        return ""
    return str(val).strip()

# -----------------------
# Interface
# -----------------------
label1 = Label("📄 Selecione a Planilha 1 (antiga):")
upload1 = FileUpload(accept='.csv', multiple=False)

label2 = Label("📄 Selecione a Planilha 2 (nova):")
upload2 = FileUpload(accept='.csv', multiple=False)

button = Button(description="🔄 Gerar Planilhas", button_style='success')
output_label = Label()

# -----------------------
# Lógica de mesclagem
# -----------------------
def escolher_valor(antigo, novo, coluna):
    antigo_s = "" if pd.isna(antigo) else str(antigo).strip()
    novo_s = "" if pd.isna(novo) else str(novo).strip()

    if not antigo_s and not novo_s:
        return ""

    if "nome" in normalize_colname(coluna) and antigo_s and novo_s:
        similaridade = SequenceMatcher(None, antigo_s.lower(), novo_s.lower()).ratio()
        if similaridade < 0.5:
            return novo_s
        return novo_s if len(novo_s) >= len(antigo_s) else antigo_s

    if not novo_s:
        return antigo_s
    return novo_s

# -----------------------
# Processamento principal
# -----------------------
def process_files(_):
    if not upload1.value or not upload2.value:
        output_label.value = "⚠️ Selecione as duas planilhas antes de continuar!"
        return

    file1 = list(upload1.value.values())[0]
    file2 = list(upload2.value.values())[0]
    df1 = pd.read_csv(StringIO(file1['content'].decode('utf-8')))
    df2 = pd.read_csv(StringIO(file2['content'].decode('utf-8')))

    # Detectar coluna de telefone (nome original)
    telefone_col = None
    for c in df1.columns:
        n = normalize_colname(c)
        if "tel" in n or "fone" in n or n == "telefone":
            telefone_col = c
            break
    if telefone_col is None:
        output_label.value = "❌ Nenhuma coluna de telefone encontrada!"
        return

    # Identificar colunas 'origem' e 'observação' pelo nome normalizado
    origem_col = find_column(df1.columns, "origem")
    obs_col = find_column(df1.columns, "observacao") or find_column(df1.columns, "observação")

    # Se 'observação' não existir, criamos com o nome 'observação' (vazio) — só para evitar erro
    if obs_col is None:
        df1["observação"] = ""
        df2["observação"] = ""
        obs_col = "observação"

    # Verificar que colunas entre df1 e df2 correspondem (ignora acentos/maiúsculas)
    if set(map(normalize_colname, df1.columns)) != set(map(normalize_colname, df2.columns)):
        output_label.value = "❌ As planilhas devem ter as mesmas colunas (mesma estrutura)."
        return

    # Padronizar telefones
    df1[telefone_col] = df1[telefone_col].astype(str).str.replace(r'\D', '', regex=True)
    df2[telefone_col] = df2[telefone_col].astype(str).str.replace(r'\D', '', regex=True)

    # Remover duplicados mantendo ultimo
    df1 = df1.drop_duplicates(subset=[telefone_col], keep='last')
    df2 = df2.drop_duplicates(subset=[telefone_col], keep='last')

    # Merge com sufixos temporários
    merged = pd.merge(df1, df2, on=telefone_col, how='outer', suffixes=('_antigo', '_novo'))
    base_cols = [c for c in df1.columns if c != telefone_col]

    final_rows = []
    altered_rows = []

    for _, row in merged.iterrows():
        final_row = {}
        final_row[telefone_col] = row[telefone_col]

        # Observação segura (prioriza nova, se existir; se nova vazia usa antiga)
        obs_antiga = safe_get_str(row, f"{obs_col}_antigo")
        obs_nova = safe_get_str(row, f"{obs_col}_novo")
        observacao_final = obs_nova if obs_nova else obs_antiga  # NOTA: '' são tratados

        houve_alteracao = False

        for col in base_cols:
            val_antigo = row.get(f"{col}_antigo", "")
            val_novo = row.get(f"{col}_novo", "")
            val_final = escolher_valor(val_antigo, val_novo, col)

            # Se é a coluna 'origem' — só então avaliar mudança e acrescentar à observação
            if origem_col and normalize_colname(col) == normalize_colname(origem_col):
                origem_antiga = safe_get_str(row, f"{col}_antigo")
                origem_nova = safe_get_str(row, f"{col}_novo")
                if origem_antiga and origem_nova and origem_antiga != origem_nova:
                    complemento = f"entrada anterior: {origem_antiga}"
                    if observacao_final:
                        # preservar observacao_final e acrescentar complemento sem apagar nada
                        observacao_final = observacao_final.rstrip().rstrip(".") + ". " + complemento
                    else:
                        observacao_final = complemento

            # Escrever valor final na coluna original
            final_row[col] = val_final

            antigo_str = safe_get_str(row, f"{col}_antigo")
            if antigo_str and val_final != antigo_str:
                houve_alteracao = True

        # Atribuir observação final (garantindo não sobrescrever com 'nan' etc.)
        final_row[obs_col] = observacao_final

        final_rows.append(final_row)

        if houve_alteracao:
            altered_rows.append(final_row)

    # Construir DataFrames finais com as colunas exatamente como em df1
    final_df = pd.DataFrame(final_rows, columns=list(df1.columns))
    alterados_df = pd.DataFrame(altered_rows, columns=list(df1.columns))

    # Relatório
    relatorio = f"""📊 RELATÓRIO DE MUDANÇAS
Planilha 1: {len(df1)} contatos
Planilha 2: {len(df2)} contatos
Final: {len(final_df)} contatos
Alterados: {len(alterados_df)}
"""

    # Salvar
    final_df.to_csv("planilha_final.csv", index=False)
    alterados_df.to_csv("planilha_alterados.csv", index=False)
    with open("relatorio_mudancas.txt", "w", encoding="utf-8") as f:
        f.write(relatorio)

    output_label.value = (
        "✅ Arquivos gerados com sucesso!\n"
        "📁 planilha_final.csv — base consolidada (sem colunas novas)\n"
        "📁 planilha_alterados.csv — registros alterados\n"
        "📄 relatorio_mudancas.txt — resumo"
    )

# conectar botão
button.on_click(process_files)
display(VBox([label1, upload1, label2, upload2, button, output_label]))
